In [2]:
# 1. Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import itertools

# 2. Load data
train = pd.read_csv("processed_data/train.csv")
val = pd.read_csv("processed_data/val.csv")
test = pd.read_csv("processed_data/test.csv")

# 3. Separate features and target
X_train = train.drop("G3_binary", axis=1)
y_train = train["G3_binary"]
X_val = val.drop("G3_binary", axis=1)
y_val = val["G3_binary"]
X_test = test.drop("G3_binary", axis=1)
y_test = test["G3_binary"]

# 4. Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

# 5. Split columns
cat_cols = [col for col in X_train.columns if any(prefix in col for prefix in 
    ['school_', 'sex_', 'address_', 'famsize_', 'Pstatus_', 'Mjob_', 'Fjob_', 
     'reason_', 'guardian_', 'schoolsup_', 'famsup_', 'paid_', 'activities_', 
     'nursery_', 'higher_', 'internet_', 'romantic_'])]
num_cols = [col for col in X_train.columns if col not in cat_cols]

X_train_wide, X_val_wide, X_test_wide = X_train[cat_cols], X_val[cat_cols], X_test[cat_cols]
X_train_deep, X_val_deep, X_test_deep = X_train[num_cols], X_val[num_cols], X_test[num_cols]

# 6. Define function to build Wide & Deep model
def build_wide_deep(hidden_layers, learning_rate=0.001, dropout_rate=0.3):
    input_wide = layers.Input(shape=(X_train_wide.shape[1],), name="wide_input")
    input_deep = layers.Input(shape=(X_train_deep.shape[1],), name="deep_input")

    # Deep branch
    deep = input_deep
    for units in hidden_layers:
        deep = layers.Dense(units, activation='relu')(deep)
        if dropout_rate > 0:
            deep = layers.Dropout(dropout_rate)(deep)

    # Concatenate wide and deep
    combined = layers.concatenate([input_wide, deep])
    output = layers.Dense(1, activation='sigmoid')(combined)

    model = keras.Model(inputs=[input_wide, input_deep], outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy', keras.metrics.AUC(name='auc')])
    return model

# 7. Hyperparameter grid (~8 models total)
param_grid = [
    {"layers":[[10]], "lr":[0.001, 0.01], "dropout":[0.0, 0.3]},
    {"layers":[[16,8]], "lr":[0.001, 0.01], "dropout":[0.0, 0.3]}
]

# Flatten param grid into list of configs
configs = []
for g in param_grid:
    for combo in itertools.product(g["layers"], g["lr"], g["dropout"]):
        configs.append({"layers": combo[0], "lr": combo[1], "dropout": combo[2]})

# 8. Train and evaluate each model
results = []
for idx, cfg in enumerate(configs, 1):
    print(f"\nTraining Model {idx} with {cfg}")
    model = build_wide_deep(hidden_layers=cfg["layers"], learning_rate=cfg["lr"], dropout_rate=cfg["dropout"])
    model.fit([X_train_wide, X_train_deep], y_train,
              validation_data=([X_val_wide, X_val_deep], y_val),
              epochs=30, batch_size=32, verbose=0,
              class_weight=class_weights_dict)

    test_probs = model.predict([X_test_wide, X_test_deep]).ravel()
    test_preds = (test_probs > 0.5).astype(int)

    acc = accuracy_score(y_test, test_preds)
    prec = precision_score(y_test, test_preds, zero_division=0)
    rec = recall_score(y_test, test_preds, zero_division=0)
    f1 = f1_score(y_test, test_preds, zero_division=0)
    auc = roc_auc_score(y_test, test_probs)

    results.append({
        "Model No.": idx,
        "Layers": cfg["layers"],
        "Learning Rate": cfg["lr"],
        "Dropout": cfg["dropout"],
        "Accuracy": round(acc, 4),
        "Precision": round(prec, 4),
        "Recall": round(rec, 4),
        "F1-Score": round(f1, 4),
        "ROC-AUC": round(auc, 4)
    })

# 9. Display results
results_df = pd.DataFrame(results)
print("\nWide & Deep Model Hyperparameter Tuning Results:")
print(results_df)

# If you want a nicer table in Jupyter
from IPython.display import display
display(results_df)



Training Model 1 with {'layers': [10], 'lr': 0.001, 'dropout': 0.0}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

Training Model 2 with {'layers': [10], 'lr': 0.001, 'dropout': 0.3}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

Training Model 3 with {'layers': [10], 'lr': 0.01, 'dropout': 0.0}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Training Model 4 with {'layers': [10], 'lr': 0.01, 'dropout': 0.3}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step

Training Model 5 with {'layers': [16, 8], 'lr': 0.001, 'dropout': 0.0}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Training Model 6 with {'layers': [16, 8], 'lr': 0.001, 'dropout': 0.3}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

Training Model 7 with {'layers': [16, 8], 'lr': 0.01, 'dropout': 0.0}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

Training Model 8 with {'layers': [16, 8], 'lr': 0.01, 'dropout': 0.3}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

Wide & Deep Model Hyperparameter Tuning Results:
   Model No.   Layers  Learning Rate  Dropout  Accuracy  Precision  Recall  \
0       

,Model No.,Layers,Learning Rate,Dropout,Accuracy,Precision,Recall,F1-Score,ROC-AUC
0,1,[10],0.001,0.0,0.6167,0.2273,0.4545,0.3030,0.6883
1,2,[10],0.001,0.3,0.5667,0.2222,0.5455,0.3158,0.6271
2,3,[10],0.010,0.0,0.6833,0.2500,0.3636,0.2963,0.6623
3,4,[10],0.010,0.3,0.6500,0.2500,0.4545,0.3226,0.7384
4,5,"[16, 8]",0.001,0.0,0.6167,0.2500,0.5455,0.3429,0.6531
5,6,"[16, 8]",0.001,0.3,0.6333,0.2800,0.6364,0.3889,0.7217
6,7,"[16, 8]",0.010,0.0,0.7833,0.3333,0.1818,0.2353,0.7384
7,8,"[16, 8]",0.010,0.3,0.6667,0.2857,0.5455,0.3750,0.6735
